Step 1: Create df of urls w/how many pages are in the topic.

Step 2: Create system for filling out those two things in new df.

## The Plan
- Gather each piece of data that we care about
- Move through each page that exists.
- End at the end of the range of pages
- Have all rows of posts added to the posts_df
- Change the row value of "Topic_scrapped" to "True" in url_df

## Imports and Function definitions

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import requests
from time import sleep
from bs4 import BeautifulSoup
import json
from time import time
from datetime import datetime
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
def get_html(url):
    return BeautifulSoup(requests.get(url).content)

In [3]:
def urls_with_numbers(forum_page_url, forum_page_num = False):
    forum_soup     = get_html(forum_page_url)     # Input URL get Soup
    
    # List of urls of topics in forum page
    topic_url_list = ['https://us.battle.net' + topic.attrs['href'] 
        for topic in forum_soup.find_all(attrs={'class': "ForumTopic"})]
    topics         = []
    count          = 0
    
    # for i in all the forum topic infos
    for forum_topic in forum_soup.find_all(attrs={'class': "ForumTopic"}):
        # Turning each individual info bit into a dict
        posts_num   = json.loads(forum_topic.attrs['data-forum-topic']
            )['lastPosition'] 
        
        # Num of posts in Topic divided by num allowed per page +1 for 1st
        topic_pages = posts_num//20 + 1 
        
        if posts_num%20 != 0 and posts_num > 20:# If there's a remainder
            topic_pages += 1                    # Add remainder page
        topics.append({
            'url'      : topic_url_list[count],
            'pages'    : topic_pages,
            'Forum_num': forum_page_num
            })
        count += 1
    return topics # Returns list of dicts


In [5]:
def validate(post_item, datum_type):
    the_type = type(post_item)
    value = 'N/a'
    if the_type == type(None):
        value = "None"
    else:
        if   datum_type == "prof_link":
            value = post_item.attrs['href']
        elif datum_type == "auth_posts":
            value = post_item.text
        elif datum_type == "auth_img":
            value = post_item.find('img').attrs['src']
        #elif datum_type == "":
        #    value = post_item.
        else:
            value = "Terra, a mistake occured & we love you. <3"
    return value

        #elif datum_type == "":
        #    value = post_item.
        
        
        
                #    topic_statuses = soup.find(attrs = {"class":
                #'TopicPost'}).attrs['data-topic']

## Getting the initial information
My first step is getting the urls and any other needed information from the 

In [ ]:
dict_list = []
b_url = 'https://us.battle.net/forums/en/overwatch/22813879/?page='
for attempt in range(5):
    try:
        for forum_num in range(9999):
            current_list = urls_with_numbers(b_url + str(forum_num),
                forum_num)
            
            dict_list += current_list
            if forum_num % 100 == 0:
                print(forum_num, 'Added', 
                    datetime.fromtimestamp(time()
                    ).strftime('%H:%M:%S'))
    except:
        print("---------")
        print('Error', attempt)
        print("---------")
        sleep(100)
    pd.DataFrame(dict_list).to_csv('./data/urls' + str(attempt) + '.csv')

### Fixing some minor mistakes

In [6]:
filterwarnings('ignore')
# I don't seem to have read_csv on this instance so I'm going 
# to settle for from_csv and ignore the warning for now.
url_df = pd.DataFrame.from_csv('./../data/urls.csv')

While fixing another issue I came upon this:

In [6]:
url_df['url'].value_counts()[55:65]

https://us.battle.net/forums/en/overwatch/topic/20761687354    2
https://us.battle.net/forums/en/overwatch/topic/20761926666    2
https://us.battle.net/forums/en/overwatch/topic/20753097396    2
https://us.battle.net/forums/en/overwatch/topic/20762036663    2
https://us.battle.net/forums/en/overwatch/topic/20761906654    2
https://us.battle.net/forums/en/overwatch/topic/20762066698    2
https://us.battle.net/forums/en/overwatch/topic/20753166205    1
https://us.battle.net/forums/en/overwatch/topic/20745706931    1
https://us.battle.net/forums/en/overwatch/topic/20752115206    1
https://us.battle.net/forums/en/overwatch/topic/20753535216    1
Name: url, dtype: int64

Just so many duplicates, but why!?  Lets take a look at a specific instance:

In [7]:
mask = url_df['url'] == 'https://us.battle.net/forums/en/overwatch/topic/20761846667'
url_df[mask]

,Forum_num,pages,url
14,0,1,https://us.battle.net/forums/en/overwatch/topi...
62,1,1,https://us.battle.net/forums/en/overwatch/topi...


ooooohhhh, when I put in the 0th and 1st forum page, I ended up essentially scraping the same page twice.  How silly of me.  Lets do a quick and easy fix.

In [8]:
mask = url_df['Forum_num'] == 0
url_df = url_df.drop([i for i in url_df[mask].index])

But there are still about 12 duplicates...

In [9]:
url_df['url'].value_counts()[10:15]

https://us.battle.net/forums/en/overwatch/topic/20753097396    2
https://us.battle.net/forums/en/overwatch/topic/20759150839    2
https://us.battle.net/forums/en/overwatch/topic/20761526007    2
https://us.battle.net/forums/en/overwatch/topic/20747304716    1
https://us.battle.net/forums/en/overwatch/topic/20759396284    1
Name: url, dtype: int64

In [10]:
mask = url_df['url'] == 'https://us.battle.net/forums/en/overwatch/topic/20752366073'
url_df[mask]

,Forum_num,pages,url
391470,7830,1,https://us.battle.net/forums/en/overwatch/topi...
391553,7832,1,https://us.battle.net/forums/en/overwatch/topi...


No obvious explaination here.  Lets look at the larger picture

In [11]:
duplicates = [
    'https://us.battle.net/forums/en/overwatch/topic/20752366073',
    'https://us.battle.net/forums/en/overwatch/topic/20759150839',    
    'https://us.battle.net/forums/en/overwatch/topic/20755047322',    
    'https://us.battle.net/forums/en/overwatch/topic/20753097396',    
    'https://us.battle.net/forums/en/overwatch/topic/20744334507',    
    'https://us.battle.net/forums/en/overwatch/topic/20747686120',    
    'https://us.battle.net/forums/en/overwatch/topic/20758737655',    
    'https://us.battle.net/forums/en/overwatch/topic/20761526007',    
    'https://us.battle.net/forums/en/overwatch/topic/20753487689',    
    'https://us.battle.net/forums/en/overwatch/topic/20760667469',    
    'https://us.battle.net/forums/en/overwatch/topic/20761766076', 
    'https://us.battle.net/forums/en/overwatch/topic/20759291345', 
    'https://us.battle.net/forums/en/overwatch/topic/20759158440'
]
for dup_url in duplicates:
    mask = url_df['url'] == dup_url
    print(url_df[mask]['Forum_num'])

391470    7830
391553    7832
Name: Forum_num, dtype: int64
391         7
66676    1333
Name: Forum_num, dtype: int64
233047    4661
233085    4662
Name: Forum_num, dtype: int64
334444    6689
335443    6709
Name: Forum_num, dtype: int64
112677    2254
488327    9767
Name: Forum_num, dtype: int64
425697    8515
429074    8582
Name: Forum_num, dtype: int64
136808    2736
136864    2738
Name: Forum_num, dtype: int64
17273    345
17800    356
Name: Forum_num, dtype: int64
300286    6006
300289    6006
Name: Forum_num, dtype: int64
43632    872
44596    892
Name: Forum_num, dtype: int64
16394    328
16395    328
Name: Forum_num, dtype: int64
77804    1556
77817    1556
Name: Forum_num, dtype: int64
103836    2077
103850    2077
Name: Forum_num, dtype: int64


Huh. This seems to be an issue to look into further once I've gathered more information.

### Adding last page.
UGH.  I made the silly mistake of forgetting the 9999th page in the forums.  I swear one day I'll get use to the range numbering system in python.

In [12]:
b_url = 'https://us.battle.net/forums/en/overwatch/22813879/?page='
last_list = urls_with_numbers(b_url + str(9999),9999)

In [15]:
last_df = pd.DataFrame(last_list)

In [16]:
url_df.tail()

,Forum_num,pages,url
499887,9998,1,https://us.battle.net/forums/en/overwatch/topi...
499888,9998,1,https://us.battle.net/forums/en/overwatch/topi...
499889,9998,1,https://us.battle.net/forums/en/overwatch/topi...
499890,9998,1,https://us.battle.net/forums/en/overwatch/topi...
499891,9998,1,https://us.battle.net/forums/en/overwatch/topi...


In [17]:
last_df.head()

,Forum_num,pages,url
0,9999,1,https://us.battle.net/forums/en/overwatch/topi...
1,9999,1,https://us.battle.net/forums/en/overwatch/topi...
2,9999,1,https://us.battle.net/forums/en/overwatch/topi...
3,9999,1,https://us.battle.net/forums/en/overwatch/topi...
4,9999,1,https://us.battle.net/forums/en/overwatch/topi...


In [18]:
# Sticking the two df's together
url_df = pd.concat([url_df,last_df])

# Resetting the index
url_df.reset_index(inplace = True)

# Renaming old index but keeping, just in case.
url_df.rename_axis({'index': 'original_index'},
                   axis = 1, inplace = True)

In [19]:
url_df.to_csv('./../data/urls_v2.csv', index = False)

## From urls to full scrapes

In [14]:
url_df = pd.DataFrame.from_csv('./../data/urls_v2.csv')

url_df['Topic_scrapped'] = False

url_df.reset_index(inplace = True)

url_df.head()#.append()

,original_index,Forum_num,pages,url,Topic_scrapped
0,48,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
1,49,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
2,50,1,238,https://us.battle.net/forums/en/overwatch/topi...,False
3,51,1,6,https://us.battle.net/forums/en/overwatch/topi...,False
4,52,1,10,https://us.battle.net/forums/en/overwatch/topi...,False


In [13]:
posts_df = pd.DataFrame(columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'statuses',
                            'topic_url',
                            'topic_page'])

#posts_df = posts_df.append({
#    'date' : 'test_teeessest'},
#    ignore_index= True)

posts_df

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url,topic_page


## Extracting the data the want from each post of the html

In [ ]:
for attempt in range(10):
    try:
        for topic_num in range(200,1001):
            if url_df.loc[topic_num,'Topic_scrapped'] == False:
                pages     = url_df.loc[topic_num,'pages'] + 1
                topic_url = url_df.loc[topic_num,'url']
                forum_page= url_df.loc[topic_num,'Forum_num']
                for page in range(1,pages):
                    
                    # Keep us updated & Defining things @ page level
                    print('Forum:', url_df.loc[topic_num,'Forum_num'],
                          'Page:', page,'/', pages - 1, 
                          "Time:",datetime.fromtimestamp(time()
                                ).strftime('%H:%M:%S'),
                          'URL:',url_df.loc[topic_num,'url'][47:])
                    soup = get_html(topic_url + '?page=' + str(page))
                    if page == 1:
                        title = soup.find(attrs={'class':'Topic-title'}).text
                    try:
                        topic_statuses = soup.find(attrs = {"class":
                        'TopicPost'}).attrs['data-topic']
                    except:
                        topic_statuses = 'No Statuses'
                    Topic_Posts = soup.find_all(attrs = {"class":
                        'TopicPost'})
                    for post in Topic_Posts:
                        post_dict = {
                            'text'      : post.find('div',
                                {"class" :'TopicPost-bodyContent'}).text,
                            'date'      : post.find('a',# Date of org post
                                {"class" :'TopicPost-timestamp'}
                                ).attrs['data-tooltip-content'],
                            'ids_dict'  : post.attrs['data-topic-post'],
                            'post_num'  : post.attrs['id'],#Post num in topic
                            'title'     : title,# Title of Topic
                            'forum_page': forum_page,# Page in the forum
                            'topic_url' : topic_url,
                            'statuses'  : topic_statuses,
                            'prof_link' : validate(post.find(attrs ={"class":
                                'Author-avatar '}), 'prof_link'),
                            'auth_posts': validate(post.find('a',
                                {"class" :'Author-posts'}),'auth_posts'),
                            'auth_img'  : validate(post.find(attrs ={"class":
                                'Author-avatar '}), 'auth_img'),
                            'topic_page': page
                            }
                        posts_df = posts_df.append(post_dict, 
                            ignore_index = True)
                    sleep(1)
                url_df.loc[topic_num,'Topic_scrapped'] = True
            else:
                pass
    except:
        print("---------")
        print('Error', attempt)
        print("---------")
        sleep(100)
    posts_df.to_csv('../data/posts_' + str(topic_num) + '.csv')
    print('SAVED AT',topic_num)


Forum: 6 Page: 1 / 1 Time: 16:30:10 URL: /20761956632
Forum: 6 Page: 1 / 4 Time: 16:30:12 URL: /20761906615
Forum: 6 Page: 2 / 4 Time: 16:30:15 URL: /20761906615
Forum: 6 Page: 3 / 4 Time: 16:30:18 URL: /20761906615
Forum: 6 Page: 4 / 4 Time: 16:30:21 URL: /20761906615
Forum: 6 Page: 1 / 1 Time: 16:30:22 URL: /20760627243
Forum: 6 Page: 1 / 1 Time: 16:30:24 URL: /20762006694
Forum: 6 Page: 1 / 1 Time: 16:30:27 URL: /20762036645
Forum: 6 Page: 1 / 1 Time: 16:30:29 URL: /20761996700
Forum: 6 Page: 1 / 3 Time: 16:30:31 URL: /20762026580
Forum: 6 Page: 2 / 3 Time: 16:30:34 URL: /20762026580
Forum: 6 Page: 3 / 3 Time: 16:30:36 URL: /20762026580
Forum: 6 Page: 1 / 1 Time: 16:30:38 URL: /20761896702
Forum: 6 Page: 1 / 1 Time: 16:30:40 URL: /20761866653
Forum: 6 Page: 1 / 6 Time: 16:30:42 URL: /20762046679
Forum: 6 Page: 2 / 6 Time: 16:30:45 URL: /20762046679
Forum: 6 Page: 3 / 6 Time: 16:30:47 URL: /20762046679
Forum: 6 Page: 4 / 6 Time: 16:30:50 URL: /20762046679
Forum: 6 Page: 5 / 6 Time: 1

Forum: 7 Page: 2 / 3 Time: 16:36:21 URL: /20762076639
Forum: 7 Page: 3 / 3 Time: 16:36:23 URL: /20762076639
Forum: 7 Page: 1 / 1 Time: 16:36:25 URL: /20762026602
Forum: 7 Page: 1 / 1 Time: 16:36:27 URL: /20761966666
Forum: 7 Page: 1 / 1 Time: 16:36:29 URL: /20761876694
Forum: 7 Page: 1 / 1 Time: 16:36:31 URL: /20761926642
Forum: 7 Page: 1 / 1 Time: 16:36:33 URL: /20761896688
Forum: 7 Page: 1 / 1 Time: 16:36:35 URL: /20762046672
Forum: 7 Page: 1 / 1 Time: 16:36:38 URL: /20761886593
Forum: 7 Page: 1 / 4 Time: 16:36:40 URL: /20761946702
Forum: 7 Page: 2 / 4 Time: 16:36:43 URL: /20761946702
Forum: 7 Page: 3 / 4 Time: 16:36:46 URL: /20761946702
Forum: 7 Page: 4 / 4 Time: 16:36:48 URL: /20761946702
Forum: 7 Page: 1 / 1 Time: 16:36:50 URL: /20761936711
Forum: 7 Page: 1 / 1 Time: 16:36:52 URL: /20762006681
Forum: 7 Page: 1 / 113 Time: 16:36:55 URL: /20759150839
Forum: 7 Page: 2 / 113 Time: 16:36:59 URL: /20759150839
Forum: 7 Page: 3 / 113 Time: 16:37:04 URL: /20759150839
Forum: 7 Page: 4 / 113

Forum: 8 Page: 1 / 1 Time: 16:46:10 URL: /20761876692
Forum: 8 Page: 1 / 1 Time: 16:46:12 URL: /20761886505
Forum: 8 Page: 1 / 1 Time: 16:46:14 URL: /20761926641
Forum: 8 Page: 1 / 1 Time: 16:46:16 URL: /20761906614
Forum: 8 Page: 1 / 4 Time: 16:46:19 URL: /20760668569
Forum: 8 Page: 2 / 4 Time: 16:46:22 URL: /20760668569
Forum: 8 Page: 3 / 4 Time: 16:46:24 URL: /20760668569
Forum: 8 Page: 4 / 4 Time: 16:46:27 URL: /20760668569
Forum: 8 Page: 1 / 1 Time: 16:46:29 URL: /20762076586
Forum: 8 Page: 1 / 1 Time: 16:46:31 URL: /20762016666
Forum: 8 Page: 1 / 1 Time: 16:46:33 URL: /20761926640
Forum: 8 Page: 1 / 1 Time: 16:46:35 URL: /20761926639
Forum: 8 Page: 1 / 1 Time: 16:46:37 URL: /20761876689
Forum: 8 Page: 1 / 1 Time: 16:46:39 URL: /20761846617
Forum: 8 Page: 1 / 1 Time: 16:46:42 URL: /20761866634
Forum: 8 Page: 1 / 1 Time: 16:46:44 URL: /20761856584
Forum: 8 Page: 1 / 1 Time: 16:46:46 URL: /20762076655
Forum: 8 Page: 1 / 149 Time: 16:46:48 URL: /20759356748
Forum: 8 Page: 2 / 149 Tim

In [19]:
number

250

In [18]:
posts_df.to_csv('../data/posts_' + str(attempt) + '.csv')

In [12]:
posts_df

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url,topic_page,auth_img
0,Welcome to the General Discussion forum! We en...,05/23/2016 09:32 PM,"{""id"":""207423780179"",""valueVoted"":0,""rank"":{""v...",post-1,None,None,Welcome to General Discussion - Please Read!,1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,None
1,"In addition to the forum Code of Conduct, here...",05/23/2016 09:33 PM,"{""id"":""207423980450"",""valueVoted"":0,""rank"":{""v...",post-2,None,None,Welcome to General Discussion - Please Read!,1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,None
2,To help us get the most of out of your feedbac...,05/23/2016 09:36 PM,"{""id"":""207423481262"",""valueVoted"":0,""rank"":{""v...",post-1,None,None,Bug Report and Technical Support Forums,1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,None
3,"Hi everyone, We've continued to see a heavy in...",01/30/2018 09:21 PM,"{""id"":""207594900856"",""valueVoted"":0,""rank"":{""v...",post-1,\n368 posts\n,None,"[Feedback Thread] Mercy Updates - Jan 30, 2018",1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,None
4,Please try to see the Valkyrie rework didn't h...,01/30/2018 09:25 PM,"{""id"":""207595100830"",""valueVoted"":0,""rank"":{""v...",post-2,\n17 posts\n,https://playoverwatch.com/en-us/career/pc/us/M...,"[Feedback Thread] Mercy Updates - Jan 30, 2018",1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,https://blzgdapipro-a.akamaihd.net/game/unlock...
5,Ban-hammer much? Surprised you guys didnt rem...,01/30/2018 09:26 PM,"{""id"":""207592902925"",""valueVoted"":0,""rank"":{""v...",post-3,\n7 posts\n,https://playoverwatch.com/en-us/career/pc/us/O...,"[Feedback Thread] Mercy Updates - Jan 30, 2018",1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,https://blzgdapipro-a.akamaihd.net/game/unlock...
6,You guys are joking with this right? The other...,01/30/2018 09:27 PM,"{""id"":""207593602686"",""valueVoted"":0,""rank"":{""v...",post-4,\n17 posts\n,https://playoverwatch.com/en-us/career/pc/us/M...,"[Feedback Thread] Mercy Updates - Jan 30, 2018",1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,https://blzgdapipro-a.akamaihd.net/game/unlock...
7,tldr: Mercy has no Ult anymore,01/30/2018 09:28 PM,"{""id"":""207593402413"",""valueVoted"":0,""rank"":{""v...",post-5,\n353 posts\n,https://playoverwatch.com/en-us/career/pc/us/A...,"[Feedback Thread] Mercy Updates - Jan 30, 2018",1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,https://blzgdapipro-a.akamaihd.net/game/unlock...
8,You took in tons of feedback and still ignored...,01/30/2018 09:29 PM,"{""id"":""207594302097"",""valueVoted"":0,""rank"":{""v...",post-6,\n504 posts\n,https://playoverwatch.com/en-us/career/pc/us/E...,"[Feedback Thread] Mercy Updates - Jan 30, 2018",1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,https://blzgdapipro-a.akamaihd.net/game/unlock...
9,01/30/2018 01:25 PMPosted by MarsuPlease try t...,01/30/2018 09:29 PM,"{""id"":""207593703439"",""valueVoted"":0,""rank"":{""v...",post-7,\n6 posts\n,https://playoverwatch.com/en-us/career/pc/us/W...,"[Feedback Thread] Mercy Updates - Jan 30, 2018",1,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,1,https://blzgdapipro-a.akamaihd.net/game/unlock...


In [118]:
url_df.loc[number,'url'][47:]

'/20761088072'

In [109]:
soup.find(attrs = {"class":
                'TopicPost'}).attrs

{'class': ['TopicPost', 'is-highly-rated', ''],
 'id': 'post-21',
 'data-topic-post': '{"id":"207592902941","valueVoted":0,"rank":{"voteUp":293,"voteDown":16},"author":{"id":"207576398916","name":"TheArchangel"}}'}

In [105]:
posts_df
#posts_df = posts_df.append({
#    'date' : 'test_teeessest'},
#    ignore_index= True)

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url,auth_img
0,Welcome to the General Discussion forum! We en...,05/23/2016 09:32 PM,"{""id"":""207423780179"",""valueVoted"":0,""rank"":{""v...",post-1,None,None,Welcome to General Discussion - Please Read!,9999,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,None
1,"In addition to the forum Code of Conduct, here...",05/23/2016 09:33 PM,"{""id"":""207423980450"",""valueVoted"":0,""rank"":{""v...",post-2,None,None,Welcome to General Discussion - Please Read!,9999,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,None
2,To help us get the most of out of your feedbac...,05/23/2016 09:36 PM,"{""id"":""207423481262"",""valueVoted"":0,""rank"":{""v...",post-1,None,None,Bug Report and Technical Support Forums,9999,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,None


In [72]:
# Get HTML and save it as Soup
title = test_soup.find(attrs={'class':'Topic-title'}).text
topic_statuses = test_soup.find(attrs = {"class" :'TopicPost'}
    ).attrs['data-topic']
Topic_Posts = test_soup.find_all(attrs = {"class" :'TopicPost'})
test_list_dicts = []
count = 0
for post in Topic_Posts:
    test_dict = {
    'text'      : post.find('div',
        {"class" :'TopicPost-bodyContent'}).text,
    'date'      : post.find('a',
        {"class" :'TopicPost-timestamp'}
        ).attrs['data-tooltip-content'],# Date of (unedited) post
    'ids_dict'  : post.attrs['data-topic-post'],# Author info & votes
    'post_num'  : post.attrs['id'],# Post number in the topic
    'title'     : title,# Title of Topic
    'forum_page': test_F_page,# Page in the forum
    'topic_url' : test_url,
    'statuses'  : topic_statuses,
    'prof_link' : validate(post.find(attrs = {"class" :
        'Author-avatar '}), 'prof_link'),
    'auth_posts': validate(post.find('a',
        {"class" :'Author-posts'}),'auth_posts'),
    'auth_img'  : validate(post.find(attrs = {"class" :
        'Author-avatar '}), 'auth_img')
    }
    test_list_dicts.append(test_dict)

pd.DataFrame(test_list_dicts, columns = ['text',
    'date', 'ids_dict', 'post_num', 'auth_posts',
    'auth_img', 'prof_link', 'title', 'forum_page',
    'statuses', 'topic_url'])

0
1
2
3
4
5
6
7
8
9
10
11
12


,text,date,ids_dict,post_num,auth_posts,auth_img,prof_link,title,forum_page,statuses,topic_url
0,"i can join a game, play for a few minutes and ...",05/23/2016 11:48 PM,"{""id"":""207422680682"",""valueVoted"":0,""rank"":{""v...",post-1,\n3 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/A...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
1,Same issue here. I'm on the west coast of the ...,05/24/2016 12:09 AM,"{""id"":""207424974255"",""valueVoted"":0,""rank"":{""v...",post-2,\n58 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/I...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
2,"Same here, has happened 3 times now",05/24/2016 12:10 AM,"{""id"":""207424374265"",""valueVoted"":0,""rank"":{""v...",post-3,\n3 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/R...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
3,Happening all over -- East Coast US here. Jus...,05/24/2016 12:13 AM,"{""id"":""207425074264"",""valueVoted"":0,""rank"":{""v...",post-4,\n1763 posts\n,None,None,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
4,Same here...,05/24/2016 12:19 AM,"{""id"":""207425474264"",""valueVoted"":0,""rank"":{""v...",post-5,\n21 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/N...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
5,"East Coast US, started the game > options > di...",05/24/2016 12:24 AM,"{""id"":""207425074284"",""valueVoted"":0,""rank"":{""v...",post-6,\n127 posts\n,None,None,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
6,"seems to just be server problems, which is exp...",05/24/2016 12:28 AM,"{""id"":""207424374309"",""valueVoted"":0,""rank"":{""v...",post-7,\n37 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/s...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
7,I finally stopped D/Cing but all I do is watch...,05/24/2016 12:31 AM,"{""id"":""207424574311"",""valueVoted"":0,""rank"":{""v...",post-8,\n31 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/T...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
8,Its launch day probably another DDOT attack,05/24/2016 12:37 AM,"{""id"":""207424974299"",""valueVoted"":0,""rank"":{""v...",post-9,\n290 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/L...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
9,"i cant even join a game, i ""join"" then gets st...",05/24/2016 12:47 AM,"{""id"":""207424574349"",""valueVoted"":0,""rank"":{""v...",post-10,\n59 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/L...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...


In [75]:
url_df.head()

,original_index,Forum_num,pages,url,Topic_scrapped
0,48,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
1,49,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
2,50,1,238,https://us.battle.net/forums/en/overwatch/topi...,False
3,51,1,6,https://us.battle.net/forums/en/overwatch/topi...,False
4,52,1,10,https://us.battle.net/forums/en/overwatch/topi...,False


In [78]:
for row_maybe, something in url_df.index:
    print(row_maybe, something)

TypeError: cannot unpack non-iterable int object

### References

In [ ]:
# THIS is for turning into the Requests library
list_of_dicts = []                                              # Instantiate empty list of dicts to fill
saver = './data/full_scrapes/Overwatch_Test_'                   # The path & starting name for saving junk

if forum_page % 100 == 0:
    save_posts(list_of_dicts, saver, forum_page) 
for topic_stats in topic_keys:         # For every title in the list
    for page in range(topic_stats[1]): # for page # in range of # of pages in the topic
        page += 1                      # Add 1 to compensate for starting at 0
        if page > 1:                   # Basically making sure it's not the first page
            URL = topic_stats[0] + '?page=' + str(page)     # This is basicslly for getting to all the pages
        else:
            URL = topic_stats[0]                            # This will be the first page

        # This chunk defines the lists of things that we want per row
        page_soup  = get_html(URL)                          # Soup of the page
        title      = page_soup.find(attrs={'class':'Topic-title'}).contents
        dates      = [e for e in page_soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents !=edt]
        words      = page_soup.find_all('div',{"class" :'TopicPost-bodyContent'})
        TopicPosts = page_soup.find_all(attrs = {"class" :'TopicPost'})
        auth_posts = [e.contents for e in page_soup.find_all('a',attrs = {'class' :'Author-posts'})]
        prof_link  = page_soup.find_all(attrs = {"class" :'Author-avatar '})
        print("forum page:", forum_page, "Posts in Topic:", topic_stats[1],"Page:",page,"URL:",URL )
        for post in range(len(words)):                           # For each post that's present
            list_of_dicts.append(turn_to_dict(post))        # Add the dict of post to list of posts
    print("forum page:", forum_page,"Topic Page:", page, "Last URL:", URL) # Sanity check
df = save_posts(list_of_dicts, saver, forum_page)               # Should both save the data & create a df to see

In [14]:
test_auth_posts_num = [int(e.find('a', attrs = {'class' :'Author-posts'}).text.strip('\n').strip(' posts')) for e in test_post_htmls]

test_dates = [e.find('a', {"class" :'TopicPost-timestamp'}).text.strip('\n').strip('\t').strip('\n') for e in test_post_htmls]

#test_dates = [e.find('a', {"class" :'TopicPost-timestamp'}).attrs['data-tooltip-content'] 
     for e in test_post_htmls]

test_words = [e.find('div',{"class" :'TopicPost-bodyContent'}).text 
    for e in test_post_htmls]

test_auth_posts = [e.find('a', {"class" :'Author-avatar '}).attrs for e in test_post_htmls]
test_auth_posts

test_prof.attrs

In [41]:
# Create the lists of desired contents
test_dates = [e.text.strip('\n').strip('\t').strip('\n') 
    for e in test_html.find_all('a', {"class" :'TopicPost-timestamp'})]

test_words = [e.text for e in test_html.find_all('div',{"class" :'TopicPost-bodyContent'})]

#TopicPosts = 
test_html.find_all(attrs = {"class" :'TopicPost'})[0].find_all('a',attrs = {'class' :'Author-posts'})

#len(test_words)
#test_html.find_all(attrs = {"class" :'TopicPost'})[1]
#auth_posts = [e.contents for e in test_html.find_all('a',attrs = {'class' :'Author-posts'})]
#prof_link  = test_html.find_all(a

[<a class="Author-posts" data-toggle="tooltip" data-tooltip-content="View Post History" href="/forums/en/overwatch/search?a=Akasha%2312113">
 3 posts
 </a>]

## Other Older Stuff

In [5]:
def save_posts(list_of_dicts, saver, forum_page):
    return pd.DataFrame(list_of_dicts, columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'stqatuses',
                            'topic_url']).to_csv(saver + str(forum_page), index=False)

In [6]:
words = []
dates = []
TopicPosts = []
title = []
forum_page = []
prof_link = []
auth_posts = []
topic_keys  = []
def turn_to_dict(post, words = words, 
                 dates = dates, TopicPosts = TopicPosts, 
                 title = title, forum_page = forum_page, 
                 prof_link = prof_link, auth_posts = auth_posts, topic_keys = topic_keys):
    post_dict = {}
    try:
        post_dict = {                               # Creation & Statement of dicts
            'text'      : words[post].contents,                     # Text of the post
            'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
            'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
            'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
            'title'     : title,                    # Title of Topic
            'forum_page': forum_page,               # Page in the forum
            'topic_url' : topic_keys[post][0]}
    except:
        pass
    
    try:
        post_dict['statuses']   = TopicPosts[post].attrs['data-topic']
    except:
        pass
    
    try:
        post_dict['prof_link']  = prof_link[post].attrs['href']
    except:
        pass
    
    try:
        post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
    except:
        pass
    
    return post_dict

In [7]:
edt = ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']

In [10]:
# THIS is for turning into the Requests library
list_of_dicts = []                                              # Instantiate empty list of DICKS to FILL UP
saver = './data/full_scrapes/Overwatch_Test_'                   # The path & starting name for saving junk

for forum_page in range(1,101):                                 # Will go through all the forum pages specified
    topic_keys = urls_with_numbers('https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page))
    if forum_page % 100 == 0:
        save_posts(list_of_dicts, saver, forum_page) 
    for topic_stats in topic_keys:                              # For every title in the list
        for page in range(topic_stats[1]):                      # for page # in range of # of pages in the topic
            page += 1                                           # Add 1 to compensate for starting at 0
            if page > 1:                                        # Basically making sure it's not the first page
                URL = topic_stats[0] + '?page=' + str(page)     # This is basicslly for getting to all the pages
            else:
                URL = topic_stats[0]                            # This will be the first page
            
            # This chunk defines the lists of things that we want per row
            page_soup  = get_html(URL)                          # Soup of the page
            title      = page_soup.find(attrs={'class':'Topic-title'}).contents
            dates      = [e for e in page_soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents !=edt]
            words      = page_soup.find_all('div',{"class" :'TopicPost-bodyContent'})
            TopicPosts = page_soup.find_all(attrs = {"class" :'TopicPost'})
            auth_posts = [e.contents for e in page_soup.find_all('a',attrs = {'class' :'Author-posts'})]
            prof_link  = page_soup.find_all(attrs = {"class" :'Author-avatar '})
            print("forum page:", forum_page, "Posts in Topic:", topic_stats[1],"Page:",page,"URL:",URL )
            for post in range(len(words)):                           # For each post that's present
                list_of_dicts.append(turn_to_dict(post))        # Add the dict of post to list of posts
        print("forum page:", forum_page,"Topic Page:", page, "Last URL:", URL) # Sanity check
df = save_posts(list_of_dicts, saver, forum_page)               # Should both save the data & create a df to see

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

In [ ]:
# Tester to see if this will continue to run within different situations
for i in range(60):
    print(datetime.fromtimestamp(time()).strftime('%H:%M:%S'))
    sleep(30)

23:43:47
23:44:17
23:44:48
23:45:18
23:45:48
23:46:18
23:46:48
23:47:18
23:47:48
23:48:18
23:48:48
23:49:18
23:49:48
23:50:18
23:50:48
23:51:18
23:51:48
23:52:18
23:52:48
23:53:18
23:53:48
23:54:18
23:54:48
23:55:18
23:55:48
23:56:18
23:56:48
23:57:18
23:57:48
23:58:18
23:58:48
23:59:18
23:59:48
00:00:18
00:00:48
00:01:18
00:01:48
00:02:18


In [ ]:
2:18

In [13]:
pd.DataFrame.from_csv('./data/urls0.csv')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,Forum_num,pages,url
0,0,1,https://us.battle.net/forums/en/overwatch/topi...
1,0,1,https://us.battle.net/forums/en/overwatch/topi...
2,0,238,https://us.battle.net/forums/en/overwatch/topi...
3,0,6,https://us.battle.net/forums/en/overwatch/topi...
4,0,10,https://us.battle.net/forums/en/overwatch/topi...
5,0,1,https://us.battle.net/forums/en/overwatch/topi...
6,0,1,https://us.battle.net/forums/en/overwatch/topi...
7,0,1,https://us.battle.net/forums/en/overwatch/topi...
8,0,1,https://us.battle.net/forums/en/overwatch/topi...
9,0,3,https://us.battle.net/forums/en/overwatch/topi...


In [24]:
def urls_with_numbers(forum_page_url, forum_page_num = False):
#current_forum_page = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
    forum_soup     = get_html(forum_page_url)                                       # Input URL get Soup
    topic_url_list = ['https://us.battle.net' + topic.attrs['href'] 
        for topic in forum_soup.find_all(attrs={'class': "ForumTopic"})]
    topic_tuples   = []                                                           # Above is list of urls of topics in forum page
    count          = 0                                                                   # Instantiating empty list and starting count
    for forum_topic in forum_soup.find_all(attrs={'class': "ForumTopic"}):      # for i in all th3e forum topic infos
        posts_num = json.loads(forum_topic.attrs['data-forum-topic']
            )['lastPosition'] # Turning each individual info bit into a dict
        topic_pages = posts_num//20 + 1                     # Number of posts in the Topic divided by num allowed per page +1 for 1st
        if posts_num%20 != 0 and posts_num > 20:            # If there's a remainder page
            topic_pages += 1                                # Add remainder page
        if forum_page_num == False:
            topic_tuples.append((topic_url_list[count],topic_pages)) # Add the url to the page amount in tuples
        else:
            topic_tuples.append((forum_page_num,topic_url_list[count],topic_pages))
        count += 1                                          # Keep track of the count
    return topic_tuples                                     # Returns list of tuples


In [54]:
def validate_prof_link(post_html):
    potential_item = post_html.find(attrs = {"class" :
        'Author-avatar '})
    value = 'N/a'
    if type(potential_item) == NoneType:
        value = "None"
    elif type(potential_item) != bs4.element.Tag:
        value = potential_item.attrs['href']
    else:
        value = "I have no idea what this is"
    return value

In [37]:

for post in test_Topic_Posts:
#    date                = post.find('a', {"class" :'TopicPost-timestamp'}).attrs['data-tooltip-content']
#    words               = post.find('div',{"class" :'TopicPost-bodyContent'}).text 
#    auth_posts_total    = post.find('a', {"class" :'Author-posts'}).text
#    data_dict           = post.attrs['data-topic-post']
#    post_num_in_topic   = post.attrs['id']
    #status              = post.attrs['data-topic']
    Auth_link           = post.find(attrs = {"class" :'Author-avatar '}).attrs['href']
    Auth_img            = post.find(attrs = {"class" :'Author-avatar '}).find('img').attrs['src']

AttributeError: 'NoneType' object has no attribute 'attrs'

In [4]:
# For use in scraper to keep things cleaner
# Using this instead of just as a dictionary will allow me to
# avoid a script-stopping error if a piece of data doesn't exist.
def save_datum(dictionary, column, datum):
    try:
        dictionary[str(column)] = datum
    except:
        print(dictionary, 'has no', column, 'data to assign')
    return

In [ ]:
test_dict = {}
save_datum(test_dict,'text'      , words[post].contents)# Text of the post
save_datum(test_dict,'date'      , dates[post].attrs['data-tooltip-content'])# Date of the (unedited) post
save_datum(test_dict,'ids_dict'  , TopicPosts[post].attrs['data-topic-post'])# Author info & votes of the post
save_datum(test_dict,'post_num'  , TopicPosts[post].attrs['id'])# Post number in the topic
save_datum(test_dict,'title'     , title)# Title of Topic
save_datum(test_dict,'forum_page', forum_page)# Page in the forum
save_datum(test_dict,'topic_url' , topic_keys[post][0])
save_datum(test_dict,'statuses'  , TopicPosts[post].attrs['data-topic'])
save_datum(test_dict,'prof_link' , prof_link[post].attrs['href'])
save_datum(test_dict,'auth_posts', auth_posts[post])  # Number of posts author has made


In [72]:
# Get HTML and save it as Soup
title = test_soup.find(attrs={'class':'Topic-title'}).text
topic_statuses = test_soup.find(attrs = {"class" :'TopicPost'}
    ).attrs['data-topic']
Topic_Posts = test_soup.find_all(attrs = {"class" :'TopicPost'})
test_list_dicts = []
count = 0
for post in Topic_Posts:
    test_dict = {
    'text'      : post.find('div',
        {"class" :'TopicPost-bodyContent'}).text,
    'date'      : post.find('a',
        {"class" :'TopicPost-timestamp'}
        ).attrs['data-tooltip-content'],# Date of (unedited) post
    'ids_dict'  : post.attrs['data-topic-post'],# Author info & votes
    'post_num'  : post.attrs['id'],# Post number in the topic
    'title'     : title,# Title of Topic
    'forum_page': test_F_page,# Page in the forum
    'topic_url' : test_url,
    'statuses'  : topic_statuses,
    'prof_link' : validate(post.find(attrs = {"class" :
        'Author-avatar '}), 'prof_link'),
    'auth_posts': validate(post.find('a',
        {"class" :'Author-posts'}),'auth_posts'),
    'auth_img'  : validate(post.find(attrs = {"class" :
        'Author-avatar '}), 'auth_img')
    }
    test_list_dicts.append(test_dict)

pd.DataFrame(test_list_dicts, columns = ['text',
    'date', 'ids_dict', 'post_num', 'auth_posts',
    'auth_img', 'prof_link', 'title', 'forum_page',
    'statuses', 'topic_url'])

0
1
2
3
4
5
6
7
8
9
10
11
12


,text,date,ids_dict,post_num,auth_posts,auth_img,prof_link,title,forum_page,statuses,topic_url
0,"i can join a game, play for a few minutes and ...",05/23/2016 11:48 PM,"{""id"":""207422680682"",""valueVoted"":0,""rank"":{""v...",post-1,\n3 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/A...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
1,Same issue here. I'm on the west coast of the ...,05/24/2016 12:09 AM,"{""id"":""207424974255"",""valueVoted"":0,""rank"":{""v...",post-2,\n58 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/I...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
2,"Same here, has happened 3 times now",05/24/2016 12:10 AM,"{""id"":""207424374265"",""valueVoted"":0,""rank"":{""v...",post-3,\n3 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/R...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
3,Happening all over -- East Coast US here. Jus...,05/24/2016 12:13 AM,"{""id"":""207425074264"",""valueVoted"":0,""rank"":{""v...",post-4,\n1763 posts\n,None,None,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
4,Same here...,05/24/2016 12:19 AM,"{""id"":""207425474264"",""valueVoted"":0,""rank"":{""v...",post-5,\n21 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/N...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
5,"East Coast US, started the game > options > di...",05/24/2016 12:24 AM,"{""id"":""207425074284"",""valueVoted"":0,""rank"":{""v...",post-6,\n127 posts\n,None,None,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
6,"seems to just be server problems, which is exp...",05/24/2016 12:28 AM,"{""id"":""207424374309"",""valueVoted"":0,""rank"":{""v...",post-7,\n37 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/s...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
7,I finally stopped D/Cing but all I do is watch...,05/24/2016 12:31 AM,"{""id"":""207424574311"",""valueVoted"":0,""rank"":{""v...",post-8,\n31 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/T...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
8,Its launch day probably another DDOT attack,05/24/2016 12:37 AM,"{""id"":""207424974299"",""valueVoted"":0,""rank"":{""v...",post-9,\n290 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/L...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
9,"i cant even join a game, i ""join"" then gets st...",05/24/2016 12:47 AM,"{""id"":""207424574349"",""valueVoted"":0,""rank"":{""v...",post-10,\n59 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/L...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...


## Extracting the test html from the test url

In [38]:
# State the Row information that we need:
test_page_quan = url_df.loc[0,'pages']
test_url       = url_df.loc[0,'url']
test_done      = url_df.loc[0,'Topic_scrapped']
test_F_page    = url_df.loc[0,'Forum_num']

# Obtain the html from the url

In [8]:
test_soup = get_html(test_url)